In [4]:
import pandas as pd
import numpy as np
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
event_df = pd.read_csv("/Users/umairabid/Downloads/Event_recommendation/event.csv")
event_df['id'] = range(1, len(event_df) + 1)

In [6]:
# Define a function to remove digits from a string
def remove_numbers(text):
    return re.sub(r'\d+', '', text).strip()

# Apply the function to the 'event_title' column
event_df['event_title'] = event_df['event_title'].apply(remove_numbers)

# Display the DataFrame to verify the changes
print(event_df.head())

                        event_title          speaker               category  \
0       Digital Transformation Week  Sheryl Sandberg      Literature/Global   
1               SXSW Music Festival        Elon Musk    Music/Entertainment   
2                     TechEd Europe        Jehan Ara  Technology/Innovation   
3  European Space Agency Conference        Elon Musk      Literature/Global   
4                        Web Summit     Marc Benioff    Music/Entertainment   

                                        description   \
0  A global event focused on Film trends, challen...   
1  A global event focused on Film trends, challen...   
2  A global event focused on Art trends, challeng...   
3  A global event focused on Art trends, challeng...   
4  A global event focused on Space trends, challe...   

                             venue  id  
0   Altice Arena, Lisbon, Portugal   1  
1    Sydney Opera House, Australia   2  
2  Fira Gran Via, Barcelona, Spain   3  
3   Altice Arena, Lisbon

In [7]:


# Reorder the columns to move 'id' to the first position
columns = ['id'] + [col for col in event_df.columns if col != 'id']
event_df = event_df[columns]

# Display the DataFrame to verify the changes
event_df.head()


id                       event_title          speaker  \
0   1       Digital Transformation Week  Sheryl Sandberg   
1   2               SXSW Music Festival        Elon Musk   
2   3                     TechEd Europe        Jehan Ara   
3   4  European Space Agency Conference        Elon Musk   
4   5                        Web Summit     Marc Benioff   

                category                                       description   \
0      Literature/Global  A global event focused on Film trends, challen...   
1    Music/Entertainment  A global event focused on Film trends, challen...   
2  Technology/Innovation  A global event focused on Art trends, challeng...   
3      Literature/Global  A global event focused on Art trends, challeng...   
4    Music/Entertainment  A global event focused on Space trends, challe...   

                             venue  
0   Altice Arena, Lisbon, Portugal  
1    Sydney Opera House, Australia  
2  Fira Gran Via, Barcelona, Spain  
3   Altice Arena, Lisbon, Portugal  
4    Austin Convention Center, USA

In [8]:
event_df.sample(5)

id                event_title             speaker  \
1824  1825            Lahore TechFest           Jehan Ara   
966    967      Mobile World Congress        Nat Friedman   
3851  3852       Startup Grind Lahore          Saad Hamid   
1863  1864          Youth Tech Summit  Shahid Javed Burki   
1181  1182  Startup Pakistan Showcase           Jehan Ara   

                         category  \
1824             Youth/Technology   
966              Space/Technology   
3851  Entrepreneurship/Technology   
1863  Entrepreneurship/Technology   
1181        E-Commerce/Technology   

                                           description   \
1824  A leading event in Pakistan focused on Technol...   
966   A global event focused on Startup trends, chal...   
3851  A global event focused on Film trends, challen...   
1863  A leading event in Pakistan focused on Cloud C...   
1181  A leading event in Pakistan focused on Blockch...   

                                           venue  
1824    Lahore University of Management Sciences  
966                Expo Center, Lahore, Pakistan  
3851  Shoreline Amphitheatre, Mountain View, USA  
1863                             NUST, Islamabad  
1181             Pearl Continental Hotel, Lahore

In [9]:
event_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4067 entries, 0 to 4066
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            4067 non-null   int64 
 1   event_title   4067 non-null   object
 2   speaker       4067 non-null   object
 3   category      4067 non-null   object
 4   description   4067 non-null   object
 5   venue         4067 non-null   object
dtypes: int64(1), object(5)
memory usage: 190.8+ KB


In [10]:
event_df.duplicated().sum()

0

In [11]:
print(event_df.columns)


Index(['id', 'event_title', 'speaker', 'category', 'description ', 'venue'], dtype='object')


In [12]:
# Joining the specified columns
event_df['tags'] = event_df.apply(lambda row: f"{row['speaker']}, {row['category']}, {row['description ']}, {row['venue']}", axis=1)

# Display the resulting DataFrame
print(event_df[['event_title', 'tags']])

                           event_title  \
0          Digital Transformation Week   
1                  SXSW Music Festival   
2                        TechEd Europe   
3     European Space Agency Conference   
4                           Web Summit   
...                                ...   
4062          Tech Conference Pakistan   
4063              Cannes Film Festival   
4064       Islamabad Robotics Workshop   
4065                   Tokyo Game Show   
4066           Machine Learning Summit   

                                                   tags  
0     Sheryl Sandberg, Literature/Global, A global e...  
1     Elon Musk, Music/Entertainment, A global event...  
2     Jehan Ara, Technology/Innovation, A global eve...  
3     Elon Musk, Literature/Global, A global event f...  
4     Marc Benioff, Music/Entertainment, A global ev...  
...                                                 ...  
4062  Saad Hamid, Fashion/Global, A global event foc...  
4063  Larry Page, Robotics/Tech

In [13]:
event_df['tags'] = event_df['tags'].apply(lambda x:x.lower())

In [14]:
event_df.head()

id                       event_title          speaker  \
0   1       Digital Transformation Week  Sheryl Sandberg   
1   2               SXSW Music Festival        Elon Musk   
2   3                     TechEd Europe        Jehan Ara   
3   4  European Space Agency Conference        Elon Musk   
4   5                        Web Summit     Marc Benioff   

                category                                       description   \
0      Literature/Global  A global event focused on Film trends, challen...   
1    Music/Entertainment  A global event focused on Film trends, challen...   
2  Technology/Innovation  A global event focused on Art trends, challeng...   
3      Literature/Global  A global event focused on Art trends, challeng...   
4    Music/Entertainment  A global event focused on Space trends, challe...   

                             venue  \
0   Altice Arena, Lisbon, Portugal   
1    Sydney Opera House, Australia   
2  Fira Gran Via, Barcelona, Spain   
3   Altice Arena, Lisbon, Portugal   
4    Austin Convention Center, USA   

                                                tags  
0  sheryl sandberg, literature/global, a global e...  
1  elon musk, music/entertainment, a global event...  
2  jehan ara, technology/innovation, a global eve...  
3  elon musk, literature/global, a global event f...  
4  marc benioff, music/entertainment, a global ev...

In [15]:
event_df = event_df[['id','event_title','tags']]

In [16]:
event_df.head()

id                       event_title  \
0   1       Digital Transformation Week   
1   2               SXSW Music Festival   
2   3                     TechEd Europe   
3   4  European Space Agency Conference   
4   5                        Web Summit   

                                                tags  
0  sheryl sandberg, literature/global, a global e...  
1  elon musk, music/entertainment, a global event...  
2  jehan ara, technology/innovation, a global eve...  
3  elon musk, literature/global, a global event f...  
4  marc benioff, music/entertainment, a global ev...

In [17]:
ps = PorterStemmer()

def stem(text):
    result = []
    for i in text.split():
        result.append(ps.stem(i))
    
    return " ".join(result)

event_df['tags']= event_df['tags'].apply(stem)

In [18]:
# max_featues are the words used frequently, we need to remove or ignore stop words of english lagnuage
cv = CountVectorizer(max_features=5000,stop_words="english")

In [19]:
# convert words into array/ vector
vectors = cv.fit_transform(event_df['tags']).toarray()

In [20]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [21]:
cv.get_feature_names_out()

array(['10', 'abdul', 'accord', 'acquisit', 'address', 'adhanom',
       'advanc', 'advers', 'agha', 'agritech', 'ahm', 'ahmed', 'ahsan',
       'ai', 'aidrus', 'aim', 'al', 'ali', 'altic', 'amazon', 'amon',
       'amphitheatre', 'amsterdam', 'andi', 'android', 'annual', 'applic',
       'applications', 'ara', 'arena', 'arfa', 'art', 'artifici', 'asad',
       'asia', 'asian', 'aslam', 'aspir', 'auditorium', 'austin',
       'australia', 'automation', 'automot', 'avail', 'avoid', 'aw',
       'awan', 'awar', 'azad', 'azure', 'badshahi', 'balkhi', 'baloch',
       'banking', 'barcelona', 'basel', 'basics', 'basit', 'bay',
       'benioff', 'berners', 'best', 'big', 'birth', 'blockchain',
       'boost', 'bootcamp', 'brin', 'bring', 'brongniart', 'bruce',
       'buena', 'build', 'burki', 'busi', 'business', 'businesses',
       'carlo', 'category', 'celebr', 'center', 'centre', 'ces',
       'chaiwala', 'challeng', 'challenges', 'china', 'chrome', 'circuit',
       'cities', 'cloud', '

In [22]:
similarity = cosine_similarity(vectors)

In [23]:
sorted(list(enumerate(similarity[0])),reverse=True, key= lambda x:x[1])[1:21]

[(3, 0.8235294117647058),
 (491, 0.8235294117647058),
 (991, 0.8140915784106943),
 (2252, 0.8140915784106943),
 (3116, 0.8140915784106943),
 (524, 0.7647058823529411),
 (844, 0.7647058823529411),
 (2078, 0.7647058823529411),
 (3797, 0.7647058823529411),
 (3913, 0.7647058823529411),
 (212, 0.7514691493021792),
 (3955, 0.7514691493021792),
 (312, 0.7130240959073808),
 (722, 0.7130240959073808),
 (772, 0.7130240959073808),
 (3786, 0.7130240959073808),
 (568, 0.7058823529411765),
 (872, 0.7058823529411765),
 (2590, 0.7058823529411765),
 (2787, 0.7058823529411765)]

In [24]:
def recommend(event):
    event_index = event_df[event_df['event_title']== event].index[0]
    distance = similarity[event_index]
    event_list = sorted(list(enumerate(distance)),reverse=True, key= lambda x:x[1])[1:21]
    
    for i in event_list:
        print(event_df.iloc[i[0]].event_title)

In [28]:
recommend('Startup Grind Lahore')

Digital Payments Pakistan
Digital Pakistan Conference
GreenTech Pakistan Expo
Cyber Security Workshop
Future of Work Pakistan
AI and Machine Learning Conference
IoT Pakistan Conference
Pakistan Game Developers Conference
Lahore Innovation Summit
Digital Pakistan Conference
Blockchain Pakistan Summit
Lahore Web . Conference
Islamabad Software Engineering Summit
Blockchain Pakistan Summit
Pakistan Open Source Summit
Pakistan Cloud Conference
DEF CON
Tech Conference Pakistan
Lahore Web . Conference
Pakistan Open Source Summit


In [29]:
def precision_at_k(recommended_events, relevant_events, k):
    """
    Calculate Precision@K.
    Parameters:
        recommended_events (list): List of recommended event titles.
        relevant_events (list): List of actual relevant event titles.
        k (int): Number of top recommendations to consider.
    Returns:
        float: Precision at K.
    """
    recommended_at_k = recommended_events[:k]
    relevant_count = sum(1 for event in recommended_at_k if event in relevant_events)
    return relevant_count / k

def recall_at_k(recommended_events, relevant_events, k):
    """
    Calculate Recall@K.
    Parameters:
        recommended_events (list): List of recommended event titles.
        relevant_events (list): List of actual relevant event titles.
        k (int): Number of top recommendations to consider.
    Returns:
        float: Recall at K.
    """
    recommended_at_k = recommended_events[:k]
    relevant_count = sum(1 for event in recommended_at_k if event in relevant_events)
    return relevant_count / len(relevant_events) if relevant_events else 0

# Example usage:
# Define a test event and its relevant (ground truth) events
test_event = "Data Science Pakistan Summit"
relevant_events = [
    "Lahore Digital Innovation Conference", "Pakistan Cloud Computing Summit", 
    "Lahore Innovation Summit", "Islamabad Tech Expo", "Future of Work Pakistan"
]

# Generate recommendations for the test event
recommended_events = [  # Replace with output from your `recommend` function
    "Lahore Digital Innovation Conference", "Pakistan Cloud Computing Summit",
    "Lahore Innovation Summit", "Islamabad Tech Expo", "Future of Work Pakistan",
    "Women in Tech Pakistan", "Islamabad Robotics Workshop", "Startup Pakistan Bootcamp"
]

# Calculate Precision@K and Recall@K
k = 5
precision = precision_at_k(recommended_events, relevant_events, k)
recall = recall_at_k(recommended_events, relevant_events, k)

print(f"Precision@{k}: {precision:.2f}")
print(f"Recall@{k}: {recall:.2f}")


Precision@5: 1.00
Recall@5: 1.00
